In [ ]:
:e ImportQualifiedPost
:e FlexibleContexts
:e BlockArguments
:e TupleSections
:e FlexibleContexts
:e OverloadedStrings
:e LambdaCase

:l Plotting.hs
:l ../src/Control/Monad/Bayes/Class.hs
:l ../src/Control/Monad/Bayes/Enumerator.hs
:l ../src/Control/Monad/Bayes/Sampler.hs
:l ../src/Control/Monad/Bayes/Weighted.hs

import Control.Monad
import Data.List
import Data.Ord
import Control.Arrow (first)
import Data.Text (pack, Text)
import Numeric.Log
import Control.Arrow (second)

# Sampling

Before discussing inference, we should understand how to sample from models. This notebook explains how to do that.

We'll start with a very simple model, namely:

In [ ]:
model :: MonadSample m => m Bool
model = bernoulli 0.7

To take a sample, do:

In [ ]:
sampleIO model

True

Or with a fixed seed:

In [ ]:
sampleIOfixed model

True

To take multiple samples, you could rerun `sampleIO` many times, but it's somewhat more in the probabilistic programming spirit to instead define a distribution over multiple draws from `model` and then just sample once, as follows:

In [ ]:
multipleDraws :: MonadSample m => m [Bool]
multipleDraws = replicateM 10 model

draws <- sampleIO multipleDraws

draws

[True,True,True,False,False,True,True,True,False,True]

We can write a function to convert a list of samples to an empirical distribution, like so:

In [ ]:


emp = toEmpirical draws

emp



[(False,0.3),(True,0.7)]

In [ ]:
plotVega $ fmap (first (pack . show))  emp

In fact, we could lean even further into the spirit of probabilistic programming, and transform `model` into a distribution over plots, and sample from that:

In [ ]:
distributionOverPlots :: MonadSample m => m VegaLiteLab -- the type of plots
distributionOverPlots = 
    plotVega . fmap (first (pack . show)) . toEmpirical 
    <$> replicateM 10 model

sampleIO distributionOverPlots


Now for a continuous distribution, consider

In [ ]:
model2 :: MonadSample m => m Double
model2 = normal 0 1

Sampling is no different to before:

In [ ]:
sampleIO model2

0.6458573516293873

And as before, to obtain multiple draws:

In [ ]:
multipleDraws2 :: MonadSample m => m [Double]
multipleDraws2 = replicateM 10 model2

draws2 <- sampleIO multipleDraws2

draws2

[0.49163648226780265,1.7865992579733017,-2.072204050546524,0.5884033340235763,-0.3023344991280155,-0.40931716377448374,0.38279913380595926,-1.120999181873208,1.6274204386623616,1.7982055358718354]

We'd like to view a histogram of samples, which in the limit of many samples should tend to the PDF of a normal distribution. Again, we could apply a histogram to the list of samples, but it's nicer to apply a `histogram` function to `multipleDraws`, to define a distribution over histograms from which we'll sample.

In [ ]:
sampleIO $ 
    plotVega . fmap (first (pack . take 6 . show)) . toEmpirical . toBins 0.05 
    <$> replicateM 100000 model2



In [ ]:
model3 = do
    p <- bernoulli 0.7
    if p then normal 0 1 else normal 3 1

In [ ]:
sampleIO $ 
    plotVega . fmap (first (pack . take 6 . show)) . toEmpirical . toBins 0.05 
    <$> replicateM 100000 model3


# Weighted Sampling

For models with factor statements, we cannot just sample. For example, consider:

In [ ]:
model4 = do
    p <- bernoulli 0.7
    out <- if p then normal 0 1 else normal 3 1
    condition (out > 0)
    return out


We could ignore the `condition` statement using `prior`, as in:

In [ ]:
sampleIO . prior $ 
    plotVega . fmap (first (pack . take 6 . show)) . toEmpirical . toBins 0.05 
    <$> replicateM 100000 model4


But obviously, we'd like to take samples from the posterior. We can do so as follows:

In [ ]:
draws4 <- sampleIO $ replicateM 10 $ runWeighted model4
draws4

[(1.7633420445268524,1.0),(2.78493875227416,1.0),(1.0459312326533174,1.0),(2.4879279652564077,1.0),(1.3958158026394436,1.0),(-1.3272681754686486,0.0),(-0.4399149407281314,0.0),(3.5174010113972534,1.0),(-0.6471909156430901,0.0),(-0.3269929580826737,0.0)]

Here, we use `runWeighted` to convert `model4` into a distribution over pairs of samples and their weights. We then sample from that. To view 

In [ ]:
sampleIO $ 
    plotVega . fmap (first (pack . take 6 . show) . second (ln . exp)) . toEmpiricalWeighted . toBinsWeighted 0.05 
    <$> replicateM 100000 (runWeighted model4)

Line 2: Use bimap
Found:
first (pack . take 6 . show) . second (ln . exp)
Why not:
bimap (pack . take 6 . show) (ln . exp)

In [ ]:
model5 :: MonadInfer m => m Double
model5 = do
    x <- normal 0 5
    (factor . Exp) (cos x)
    return x

sampleIO $ 
    plotVega . fmap (first (pack . take 6 . show) . second (ln . exp)) . toEmpiricalWeighted . toBinsWeighted 0.05 
    <$> replicateM 100000 (runWeighted model5)

Line 8: Use bimap
Found:
first (pack . take 6 . show) . second (ln . exp)
Why not:
bimap (pack . take 6 . show) (ln . exp)

In [ ]:
sampleIO $ 
    plotVega . fmap (first (pack . take 6 . show)) . toEmpirical . toBins 0.05 
    <$> replicateM 100000 model3